# Data Wrangling dos Indicadores Municipais do RJ

Este notebook realiza as seguintes etapas:

- Seleção dos municípios do estado do Rio de Janeiro (RJ)
- Escolha das variáveis de interesse dos indicadores IFGF e IFDM
- Transformação dos dados do formato wide para long (empilhamento)
- Classificação dos municípios conforme faixas dos indicadores
- Consolidação dos dados em um único painel para análise comparativa

In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype

# IFDM

In [89]:
import pandas as pd

class DataWrangler:
    """
    Classe utilitária para manipulação e transformação de DataFrames de indicadores municipais.
    
    Métodos:
        - convert_year(x: str) -> str: Extrai o ano de uma string.
        - filter_state(df: pd.DataFrame, state_name: str = 'RJ') -> pd.DataFrame: Filtra municípios por estado.
        - select_columns(df: pd.DataFrame) -> pd.DataFrame: Seleciona colunas relevantes para IFDM.
        - stack_data(df: pd.DataFrame, value_name: str = 'IFDM') -> pd.DataFrame: Transforma dados wide para long.
        - pipeline(self) -> pd.DataFrame: Executa o pipeline completo de transformação.
    """

    @staticmethod
    def convert_year(x: str) -> str:
        """
        Extrai o ano de uma string no formato '... 2013', retornando '2013'.
        
        Args:
            x (str): String contendo o ano.
        Returns:
            str: Ano extraído.
        """
        return x.split(' ')[-1]

    @staticmethod
    def filter_state(df: pd.DataFrame, state_name: str = 'RJ') -> pd.DataFrame:
        """
        Filtra o DataFrame para conter apenas linhas do estado especificado.
        
        Args:
            df (pd.DataFrame): DataFrame de entrada.
            state_name (str): Sigla do estado a filtrar.
        Returns:
            pd.DataFrame: DataFrame filtrado.
        """
        return df[df['SIGLA_UF'].str.contains(state_name, case=False)].reset_index(drop=True)

    @staticmethod
    def select_columns(df: pd.DataFrame) -> pd.DataFrame:
        """
        Seleciona colunas relacionadas ao IFDM, código e nome do município.
        
        Args:
            df (pd.DataFrame): DataFrame de entrada.
        Returns:
            pd.DataFrame: DataFrame apenas com as colunas selecionadas.
        """
        return df.filter(regex='^IFDM|COD_MUNIC|NOME_MUNIC')
    
    @staticmethod
    def classify_ifdm(valor):
        if valor >= 0.8:
            return 'Alto'
        elif valor >= 0.6:
            return 'Moderado'
        elif valor >= 0.4:
            return 'Baixo'
        else:
            return 'Crítico'
    
    @staticmethod
    def classify_idfm_by_range(df, target, label):
        """
        Classifica os valores do IFDM em categorias baseadas em intervalos.
        
        Args:
            df (pd.DataFrame): DataFrame contendo a coluna 'IFDM'.
            label (str): Nome da coluna de classificação.
        Returns:
            pd.DataFrame: DataFrame com a coluna de classificação adicionada.
        """
        df[label] = df[target].apply(DataWrangler.classify_ifdm)
        return df
        

    @staticmethod
    def stack_data(df: pd.DataFrame, value_name: str = 'IFDM') -> pd.DataFrame:
        """
        Transforma o DataFrame do formato wide para long (empilhado).
        
        Args:
            df (pd.DataFrame): DataFrame de entrada.
            value_name (str): Nome da coluna de valores.
        Returns:
            pd.DataFrame: DataFrame empilhado.
        """
        df_stack = df.melt(id_vars=['COD_MUNIC', 'NOME_MUNIC'],
                           value_name=value_name, var_name='Ano')
        df_stack['Ano'] = df_stack['Ano'].apply(DataWrangler.convert_year)
        return df_stack

    def pipeline(self, df, value_name) -> pd.DataFrame:
        """
        Executa o pipeline de filtragem, seleção e transformação dos dados.
        
        Returns:
            pd.DataFrame: DataFrame transformado.
        """
        return (df
                .pipe(self.filter_state, state_name='RJ')
                .pipe(self.select_columns)
                .pipe(self.stack_data, value_name=value_name)
                .pipe(self.classify_idfm_by_range, target=value_name, label=f'{value_name}_Classificacao')
            )


In [74]:
# Carregar dados IFDM
df_ifdm = pd.read_excel('../data/Serie-Historica-IFDM-2013-a-2023.xlsx', na_values=['nd'])
df_ifdm_educacao = pd.read_excel('../data/Serie-Historica-IFDM-2013-a-2023.xlsx', na_values=['nd'], sheet_name=1)
df_ifdm_saude = pd.read_excel('../data/Serie-Historica-IFDM-2013-a-2023.xlsx', na_values=['nd'], sheet_name=2)
df_ifdm_renda = pd.read_excel('../data/Serie-Historica-IFDM-2013-a-2023.xlsx', na_values=['nd'], sheet_name=3)


Filtrar municípios do RJ e selecionar variáveis de interesse do IFDM.

In [ ]:
data_wrangler = DataWrangler()

df_ifdm_long = data_wrangler.pipeline(df_ifdm, value_name='IFDM')
df_ifdm_educacao_long = data_wrangler.pipeline(df_ifdm_educacao, value_name='IFDM_Educacao')
df_ifdm_saude_long = data_wrangler.pipeline(df_ifdm_saude, value_name='IFDM_Saude')
df_ifdm_renda_long = data_wrangler.pipeline(df_ifdm_renda, value_name='IFDM_Renda')

## Consolidar os dados

In [99]:
from functools import reduce

dfs = [
    df_ifdm_long,
    df_ifdm_educacao_long,
    df_ifdm_saude_long,
    df_ifdm_renda_long
]

df_merged = reduce(
    lambda left, right: pd.merge(
        left, right, on=['COD_MUNIC', 'NOME_MUNIC', 'Ano'], how='outer'
    ),
    dfs
)


In [102]:
df_merged.head()

,COD_MUNIC,NOME_MUNIC,Ano,IFDM,IFDM_Classificacao,IFDM_Educacao,IFDM_Educacao_Classificacao,IFDM_Saude,IFDM_Saude_Classificacao,IFDM_Renda,IFDM_Renda_Classificacao
0,330010,Angra dos Reis,2013,0.6387,Moderado,0.3745,Crítico,0.6807,Moderado,0.8609,Alto
1,330010,Angra dos Reis,2014,0.6531,Moderado,0.3917,Crítico,0.6891,Moderado,0.8785,Alto
2,330010,Angra dos Reis,2015,0.6385,Moderado,0.4004,Baixo,0.6887,Moderado,0.8264,Alto
3,330010,Angra dos Reis,2016,0.5974,Baixo,0.363,Crítico,0.6716,Moderado,0.7575,Moderado
4,330010,Angra dos Reis,2017,0.5897,Baixo,0.3862,Crítico,0.6629,Moderado,0.7201,Moderado


# CADUNICO

In [104]:
df_pbf = pd.read_csv('../data/pessoas_PBF22_06_2025.csv', encoding='latin1')
df_pbf

,Código,Unidade Territorial,UF,Referência,Pessoas beneficiárias no PBF (até Out/2021),Pessoas beneficiárias no PBF (a partir de Mar/2023)
0,330010,ANGRA DOS REIS,RJ,08/2023,NaN,48099.0
1,330010,ANGRA DOS REIS,RJ,04/2023,NaN,49156.0
2,330010,ANGRA DOS REIS,RJ,06/2023,NaN,49113.0
3,330010,ANGRA DOS REIS,RJ,11/2024,NaN,45570.0
4,330010,ANGRA DOS REIS,RJ,03/2024,NaN,45230.0
...,...,...,...,...,...,...
11219,330630,VOLTA REDONDA,RJ,04/2016,34993.0,NaN
11220,330630,VOLTA REDONDA,RJ,07/2014,34375.0,NaN
11221,330630,VOLTA REDONDA,RJ,05/2016,34740.0,NaN
11222,330630,VOLTA REDONDA,RJ,02/2016,35668.0,NaN
